In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os


In [2]:
#the folder that contains the planes
base = 'train/Task1/full-configurations-templates'

#created 3 vectors that contains the coordinates for each bowling pin by giving a bounding bow for the red part
coordinates_t1 = [[622, 314, 66, 70], [497, 205, 63, 61], [762, 206, 55, 60],
                  [426, 138, 53, 50], [639, 140, 48, 49], [855, 139, 46, 51],
                  [374, 93, 41, 42], [556, 91, 39, 44], [739, 91, 40, 45],
                  [927, 91, 39, 45]]

coordinates_t2 = [[590, 311, 66, 70], [472, 210, 52, 55], [724, 209, 53, 54],
                  [391, 143, 49, 47], [600, 144, 48, 46], [805, 142, 44, 47],
                  [339, 98, 42, 40], [517, 98, 38, 40], [691, 96, 37, 44], [865, 98, 38, 42]]

coordinates_t3 = [[601, 314, 64, 68], [470, 201, 58, 55], [745, 206, 50, 54],
                  [399, 135, 42, 45], [613, 132, 44, 47], [831, 134, 43, 48],
                  [339, 86, 40, 42], [524, 81, 40, 45], [711, 78, 34, 48],
                  [895, 83, 35, 46]]

In [3]:

#detect if the image is on the plane based on the color channels of a part of a image(the table)
def get_lane(t1, t2, t3, img):
    #crop a part of table and compute mean for each color channel
    dist = []
    t1 = t1[600:, 300:-300]
    mean_t1 = np.mean(t1, axis=(0,1))
    t2 = t2[600:, 300:-300]
    mean_t2 = np.mean(t2, axis=(0, 1))
    t3 = t3[600:, 300:-300]
    mean_t3 = np.mean(t3, axis=(0, 1))
    img = img[600:, 300:-300]
    mean_img = np.mean(img, axis=(0, 1))

    dist.append(np.sum((mean_t1 - mean_img) ** 2))
    dist.append(np.sum((mean_t2 - mean_img) ** 2))
    dist.append(np.sum((mean_t3 - mean_img) ** 2))

    return np.argmax(np.array(dist))

In [4]:
#detect if a photo is on lane 1 or 3 by comparing the down-right corner of the image with lane 1 and lane 2 using L2 norm
def get_lane_1_3(t1, t3, img):
    #crop the right down corner
    img_t1 = t1[600:-50,-400:]
    img_t3 = t3[600:-50,-400:]
    img = img[600:-50,-400:]

    #calculate error and similarity for lane1
    errorL2_t1 = cv.norm(img_t1, img, cv.NORM_L2)
    similarity_1 = 1 - errorL2_t1 / (120 * 400)
    #calculate error and similarity for lane3
    errorL2_t3 = cv.norm(img_t3, img, cv.NORM_L2)
    similarity_3 = 1 - errorL2_t3 / (120 * 400)
    if similarity_1 >= similarity_3:
        return 1
    else:
        return 3

In [5]:
#read the lanes and set the paths
t1 = base + '/' + 'lane1.jpg'
img_t1 = cv.imread(t1)

t2 = base + '/' + 'lane2.jpg'
img_t2 = cv.imread(t2)

t3 = base + '/' + 'lane3.jpg'
img_t3 = cv.imread(t3)

base_folder_images = 'train/Task1'

output = []

test_folder_images = '../test/Task1'


In [6]:
#iterate through images
#get the lane for each image
#use hsv in order to get only the red patches
#once got the lane, iterate through the coordinates listed above
#check if in the bounding boxes that are constructed using the coordinates above there is a red path
#compute the largest contour that contains red
#check if the area of red/area of the bounding box created by coordinates is greater than 0.4
#if yes, there is a pin in that position
for fname in os.listdir(test_folder_images):
    complete_filename = test_folder_images + '/' + fname
    if complete_filename[-4:] == '.jpg':
        #get the photo and the lane of the photo
        img = cv.imread(complete_filename)
        lane = get_lane(img_t1, img_t2, img_t3, img)
        if lane == 2:
            #extract the red regions
            hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

            hsv_low = np.array([0, 150, 61], np.uint8)
            hsv_high = np.array([179, 255, 255], np.uint8)
            #create mask based on hsv colors
            mask = cv.inRange(hsv, hsv_low, hsv_high)
            res = cv.bitwise_and(img, img, mask=mask)
            #use kernal and dilation to smooth the mask
            kernel = np.ones((2, 2), np.uint8)
            dilation = cv.dilate(mask, kernel, iterations=2)

            popici = 0
            popici_pozitii = []
            #iterate through coordinates
            for i, coordinate in enumerate(coordinates_t2):
                #get patches where it might be a pin
                crop = dilation[coordinate[1]:coordinate[1] + coordinate[3],
                       coordinate[0]:coordinate[0] + coordinate[2]]
                #get contours
                contours, hierarchy = cv.findContours(crop, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                if len(contours) > 0:
                    #if we get a contour and the ration between contour and the patch area is bigger than 0.4, there is a pin
                    c = max(contours, key=cv.contourArea)
                    x, y, w, h = cv.boundingRect(c)
                    if (w * h) / (crop.shape[0] * crop.shape[1]) > 0.4:
                        #count pin and add position
                        popici += 1
                        popici_pozitii.append(i)
                        #plt.imshow(crop)
                        #plt.show()
            #print(2, complete_filename, popici, popici_pozitii)
            output.append([fname, popici, popici_pozitii])
        else:
            #extract the red regions
            hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
            #create mask based on hsv colors
            hsv_low = np.array([160, 100, 20], np.uint8)
            hsv_high = np.array([179, 255, 255], np.uint8)

            mask = cv.inRange(hsv, hsv_low, hsv_high)
            res = cv.bitwise_and(img, img, mask=mask)
            #use kernal and dilation to smooth the mask
            kernel = np.ones((2, 2), np.uint8)
            dilation = cv.dilate(mask, kernel, iterations=2)



            if get_lane_1_3(img_t1, img_t3, img) == 1:
                popici = 0
                popici_pozitii = []
                for i, coordinate in enumerate(coordinates_t1):
                    #iterate through coordinates
                    #get patches where it might be a pin

                    crop = dilation[coordinate[1]:coordinate[1] + coordinate[3],
                           coordinate[0]:coordinate[0] + coordinate[2]]
                    contours, hierarchy = cv.findContours(crop, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                    if len(contours) > 0:
                    #if we get a contour and the ration between contour and the patch area is bigger than 0.4, there is a pin

                        c = max(contours, key=cv.contourArea)
                        x, y, w, h = cv.boundingRect(c)
                        if (w * h) / (crop.shape[0] * crop.shape[1]) > 0.4:
                            popici += 1
                            popici_pozitii.append(i)
                            # plt.imshow(crop)
                            # plt.show()
                #print(1, complete_filename, popici, popici_pozitii)
                output.append([fname, popici, popici_pozitii])
            else:
                popici = 0
                popici_pozitii = []
                for i, coordinate in enumerate(coordinates_t3):
                #iterate through coordinates
                #get patches where it might be a pin
                    crop = dilation[coordinate[1]:coordinate[1] + coordinate[3],
                           coordinate[0]:coordinate[0] + coordinate[2]]
                    contours, hierarchy = cv.findContours(crop, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                    if len(contours) > 0:
                        #if we get a contour and the ration between contour and the patch area is bigger than 0.4, there is a pin
                        c = max(contours, key=cv.contourArea)
                        x, y, w, h = cv.boundingRect(c)
                        if (w * h) / (crop.shape[0] * crop.shape[1]) > 0.4:
                            popici += 1
                            popici_pozitii.append(i)
                            # plt.imshow(crop)
                            # plt.show()
                #print(3, complete_filename, popici, popici_pozitii)
                output.append([fname, popici, popici_pozitii])



In [7]:
os.mkdir('../submission_files/Task1')
base = '../submission_files/Task1'
for line in output:
    #get file name, number of pins and the positions
    file = line[0]
    number = line[1]
    positions = line[2]
    #create the predicted file and add the output
    file_name = base + '/' + file[:-4] + '_predicted.txt'
    popice_pos = ""
    for i in range(number):
        popice_pos = popice_pos + str(positions[i]+1) + '\n'

    f = open(file_name, "w")
    if number == 0:
        #if there is no pins, add only the number=0
        f.write('0')
    else:
        #else add also the positions of pins
        f.write(str(number) + '\n' + popice_pos)

    f.close()